In [80]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# For data preprocess
import numpy as np
import csv
import os

myseed = 5  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [81]:
from tensorboardX import SummaryWriter 
#SummaryWriter 是一個類別，包含這套件的所有功能。

writer = SummaryWriter(comment='test')
#在預設資料夾後面加上註解 檔名變為：'runs/Aug20-obov01-3xLR'

In [82]:
if torch.cuda.is_available():
  print("is available")

is available


In [83]:
class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.n1 = nn.Linear(1,32)
    self.n2 = nn.Linear(32,2)
    self.ac = nn.ReLU()
    self.criterion = nn.CrossEntropyLoss()
  def forward(self, x):
    x = self.n1(x)
    x = self.ac(x)
    x = self.n2(x)
    return x

  def cal_loss(self, pred, target):
    ''' Calculate loss '''
    # TODO: you may implement L1/L2 regularization here
    
    return self.criterion(pred, target)
  

In [84]:
def train(tr_set, dv_set, model, config, device):
    ''' DNN training '''

    n_epochs = config['n_epochs']  # Maximum number of epochs

    # Setup optimizer
    optimizer = getattr(torch.optim, config['optimizer'])(
        model.parameters(), **config['optim_hparas'])

    min_loss = 1000.
    loss_record = {'train': [], 'dev': []}      # for recording training loss
    early_stop_cnt = 0
    epoch = 0
    iter_idx = 0
    while epoch < n_epochs:
        model.train()                           # set model to training mode
        for x, y in tr_set:                     # iterate through the dataloader
            optimizer.zero_grad()               # set gradient to zero
            x, y = x.to(device), y.to(device)   # move data to device (cpu/cuda)
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
            mse_loss.backward()                 # compute gradient (backpropagation)
            optimizer.step()                    # update model with optimizer
            loss_record['train'].append(mse_loss.detach().cpu().item())
            writer.add_scalar('myscalar', mse_loss.detach().cpu().item(), iter_idx)
            iter_idx+=1
        # After each epoch, test your model on the validation (development) set.
        dev_mse = dev(dv_set, model, device)
        if dev_mse < min_loss:
            # Save model if your model improved
            min_loss = dev_mse
            print('Saving model (epoch = {:4d}, loss = {:.4f})'
                .format(epoch + 1, min_loss))
            # torch.save(model.state_dict(), config['save_path'])  # Save model to specified path
            early_stop_cnt = 0
        else:
            early_stop_cnt += 1

        epoch += 1
        loss_record['dev'].append(dev_mse)
        if early_stop_cnt > config['early_stop']:
            # Stop training if your model stops improving for "config['early_stop']" epochs.
            break

    print('Finished training after {} epochs'.format(epoch))
    return min_loss, loss_record

In [85]:
def dev(dv_set, model, device):
    model.eval()                                # set model to evalutation mode
    total_loss = 0
    
    for x, y in dv_set:                         # iterate through the dataloader
        x, y = x.to(device), y.to(device)       # move data to device (cpu/cuda)
        with torch.no_grad():                   # disable gradient calculation
            pred = model(x)                     # forward pass (compute output)
            mse_loss = model.cal_loss(pred, y)  # compute loss
        total_loss += mse_loss.detach().cpu().item() * len(x)  # accumulate loss
        
    
    total_loss = total_loss / len(dv_set.dataset)              # compute averaged loss

    return total_loss



In [86]:


def get_device():
    ''' Get device (if GPU is available, use GPU) '''
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()                 # get the current available device ('cpu' or 'cuda')
# os.makedirs('models', exist_ok=True)  # The trained model will be saved to ./models/
target_only = True                    # TODO: Using 40 states & 2 tested_positive features

# TODO: How to tune these hyper-parameters to improve your model's performance?
config = {
    'n_epochs': 3000,                # maximum number of epochs
    'batch_size': 512,               # mini-batch size for dataloader
    'optimizer': 'Adam',              # optimization algorithm (optimizer in torch.optim)
    'optim_hparas': {                # hyper-parameters for the optimizer (depends on which optimizer you are using)
        # 'lr': 0.001,                 # learning rate of SGD
        # 'momentum': 0.9              # momentum for SGD
    },
    'early_stop': 200,               # early stopping epochs (the number epochs since your model's last improvement)
    'save_path': 'models/model.pth'  # your model will be saved here
}

In [87]:
model = Model().to(device)  # Construct model and move to device

In [88]:
class RandomDataset(Dataset):
    ''' Dataset for loading and preprocessing the COVID19 dataset '''
    def __init__(self,
                 data,
                 mode='train'):
        self.mode = mode

        target = data[1]
        data = data[0]
        
        # Splitting training data into train & dev sets
        if mode == 'train':
            indices = [i for i in range(len(data)) if i % 10 != 0]
        elif mode == 'dev':
            indices = [i for i in range(len(data)) if i % 10 == 0]
        
        # Convert data into PyTorch tensors
        self.data = data[indices].type(torch.FloatTensor)
        # self.data = torch.FloatTensor(data[indices])
        self.target = target[indices].type(torch.FloatTensor)
        # self.target = torch.FloatTensor(target[indices])
        
        self.dim = self.data.shape[1]

        print('Finished reading the {} set of COVID19 Dataset ({} samples found, each dim = {})'
              .format(mode, len(self.data), self.dim))

    def __getitem__(self, index):
        # Returns one sample at a time
        if self.mode in ['train', 'dev']:
            # For training
            return self.data[index], self.target[index]
        else:
            # For testing (no target)
            return self.data[index]

    def __len__(self):
        # Returns the size of the dataset
        return len(self.data)

In [89]:
# dataset = RandomDataset((tr_x,tr_y), 'train', target_only=False)
# a,b=dataset[0]
# print(a,b)

In [90]:
def prep_dataloader(data, mode, batch_size):
    ''' Generates a dataset, then is put into a dataloader. '''
    dataset = RandomDataset(data, mode=mode)  # Construct dataset
    dataloader = DataLoader(
        dataset, batch_size,
        shuffle=(mode == 'train'))                            # Construct dataloader
    return dataloader

In [91]:
def isUgly(n: int) -> bool:
    def recursion(n):
        if n==1:
            return 1
        elif n%2==0:
            return recursion(n//2)
        elif n%3==0:
            return recursion(n//3)
        elif n%5==0:
            return recursion(n//5)
        else:
            return n
    return recursion(n)==1 if n!=0 else False

In [92]:
tr_x = torch.unsqueeze(torch.tensor(list(range(65535))),1) 
tr_y = []
for i in tr_x:
    if isUgly(i.item()):
        
        tr_y.append([0,1])
    else:
        # print(i.item())
        tr_y.append([1,0])
tr_y = torch.FloatTensor(tr_y)




tr_set = prep_dataloader((tr_x,tr_y), 'train', config['batch_size'])
dv_set = prep_dataloader((tr_x,tr_y), 'dev', config['batch_size'])



Finished reading the train set of COVID19 Dataset (58981 samples found, each dim = 1)
Finished reading the dev set of COVID19 Dataset (6554 samples found, each dim = 1)


In [93]:
# for x in tr_set:
#   print(x[0])

In [94]:
model_loss, model_loss_record = train(tr_set, dv_set, model, config, device)

Saving model (epoch =    1, loss = 30.8009)
Saving model (epoch =    8, loss = 11.7298)
Saving model (epoch =    9, loss = 7.7608)
Saving model (epoch =   10, loss = 1.6537)
Saving model (epoch =   11, loss = 0.5873)
Saving model (epoch =   12, loss = 0.5233)
Saving model (epoch =   54, loss = 0.4664)
Saving model (epoch =   88, loss = 0.3636)
Saving model (epoch =  197, loss = 0.3042)
Saving model (epoch =  230, loss = 0.1975)
Saving model (epoch =  339, loss = 0.1892)
Saving model (epoch =  429, loss = 0.1782)
Saving model (epoch =  479, loss = 0.1342)
Saving model (epoch =  596, loss = 0.1302)
Saving model (epoch =  599, loss = 0.1266)
Saving model (epoch =  662, loss = 0.1253)
Saving model (epoch =  698, loss = 0.1157)
Saving model (epoch =  762, loss = 0.1045)
Saving model (epoch =  906, loss = 0.1019)
Finished training after 1107 epochs


In [79]:
for name, param in model.named_parameters():
    print(name,param)

n1.weight Parameter containing:
tensor([[ 6.1239e-01],
        [-7.4778e-01],
        [ 2.6735e-01],
        [ 4.7764e-01],
        [ 7.6740e-01],
        [-7.6671e-01],
        [-6.7122e-01],
        [ 3.0821e-01],
        [ 1.2655e-01],
        [ 6.9698e-01],
        [-7.2747e-04],
        [-2.1005e-04],
        [-8.5518e-01],
        [ 2.5731e-01],
        [ 3.5271e-01],
        [-9.2792e-04]], device='cuda:0', requires_grad=True)
n1.bias Parameter containing:
tensor([-0.2146,  0.3003, -3.4286,  1.4309,  1.1867, -0.4771, -0.9186,  1.1366,
         1.0557, -0.5841, -1.3510, -0.7570, -0.4722,  0.6501,  0.1603, -0.4100],
       device='cuda:0', requires_grad=True)
n2.weight Parameter containing:
tensor([[-0.2125, -0.0058,  0.1775,  0.1080,  0.2236, -0.2196, -0.0308,  0.2245,
         -0.8865, -0.0778, -0.1659,  0.1848,  0.1898,  0.1692,  0.1868, -0.0123],
        [ 0.1628, -0.0629,  0.1738, -0.0078, -0.0403, -0.1243,  0.1331,  0.1200,
          1.0865, -0.1525, -0.1921,  0.1681, -0.138